In [ ]:
import os

import astropy.coordinates as coord
coord.galactocentric_frame_defaults.set('v4.0')
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.integrate as gi
import gala.potential as gp
from gala.units import galactic
from scipy.special import logsumexp

from pyia import GaiaData
import theano
theano.config.gcc.cxxflags = "-fbracket-depth=16000" # default is 256
import pymc3 as pm
import theano.tensor as tt
import exoplanet as xo

pm.__version__

Expected mean disk velocity in ICRS:

In [ ]:
xtmp = coord.CartesianRepresentation(100, 0, 0, unit=u.km)
vlsr = coord.CartesianDifferential(-(np.array([12.9, 245.6, 7.78]) - np.array([0, 234, 0])) * u.km/u.s)
vthin = coord.SkyCoord(xtmp.with_differentials(vlsr), frame='galactic').icrs.velocity.d_xyz.value

xtmp = coord.CartesianRepresentation(100, 0, 0, unit=u.km)
vlsr = coord.CartesianDifferential(-(np.array([12.9, 245.6, 7.78]) - np.array([0, 200, 0])) * u.km/u.s)
vthik = coord.SkyCoord(xtmp.with_differentials(vlsr), frame='galactic').icrs.velocity.d_xyz.value

xtmp = coord.CartesianRepresentation(100, 0, 0, unit=u.km)
vsun = coord.CartesianDifferential(-np.array([12.9, 245.6, 7.78]) * u.km/u.s)
vhalo = coord.SkyCoord(xtmp.with_differentials(vsun), frame='galactic').icrs.velocity.d_xyz.value

vthin, vthik, vhalo

In [ ]:
pc_mas_yr_per_km_s = (1 * u.km/u.s).to(u.pc*u.mas/u.yr, u.dimensionless_angles()).value
km_s_per_pc_mas_yr = 1 / pc_mas_yr_per_km_s

In [ ]:
g = GaiaData(os.path.expanduser('~/data/GaiaDR2/rv-good-plx.fits'))

In [ ]:
the_og = g[g.source_id == 1490845584382687232]
v0 = the_og.get_skycoord().velocity.d_xyz.value.T[0]
sigma_0 = 0.6  # km/s
v0

In [ ]:
def get_tangent_basis(ra, dec):
    """
    row vectors are the tangent-space basis at (alpha, delta, r)
    ra, dec in radians
    """
    M = np.array([
        [-np.sin(ra), np.cos(ra), 0.],
        [-np.sin(dec)*np.cos(ra), -np.sin(dec)*np.sin(ra), np.cos(dec)],
        [np.cos(dec)*np.cos(ra), np.cos(dec)*np.sin(ra), np.sin(dec)]
    ])
    return M

In [ ]:
class UniformSpaceDensity(pm.Continuous):

    def __init__(self, rlim, **kwargs):
        """A uniform space density prior over a distance, r, between (0, rlim)
        """

        self.rlim = float(rlim)
        assert (self.rlim > 0)
        self._fac = np.log(3.) - 3 * np.log(self.rlim)

        shape = kwargs.get("shape", None)
        if shape is None:
            testval = 0.5 * self.rlim
        else:
            testval = 0.5 * self.rlim + np.zeros(shape)
        kwargs["testval"] = kwargs.pop("testval", testval)
        super(UniformSpaceDensity, self).__init__(**kwargs)

    def _random(self, size=None):
        uu = np.random.uniform(size=size)
        return np.cbrt(uu) * self.rlim

    def random(self, point=None, size=None):
        return generate_samples(
            self._random,
            dist_shape=self.shape,
            broadcast_shape=self.shape,
            size=size,
        )

    def logp(self, value):
        return 2 * tt.log(tt.as_tensor_variable(value)) + self._fac

In [ ]:
hip = at.QTable.read('/Users/apricewhelan/data/GaiaDR2/gaia-hipparcos.fits.gz')
hip = hip[~np.isfinite(hip['radial_velocity'])]
hip.rename_column('hip', 'HIP')

xhip = at.Table.read('/Users/apricewhelan/data/Hipparcos/xhip.fits')
xhip.meta = None

pulkovo = at.Table.read('/Users/apricewhelan/data/Hipparcos/Pulkovo.fits')
pulkovo.meta = None
pulkovo.remove_columns(['eRV', '_RA', '_DE', 'recno'])
pulkovo['q_RV'] = 3
xhip = at.vstack((xhip, pulkovo[~np.isin(pulkovo['HIP'], xhip['HIP'])]))

hip = at.join(hip, xhip, keys='HIP', 
              uniq_col_name='{col_name}{table_name}',
              table_names=['', '_2'])
hip = hip[np.isfinite(hip['RV']) & np.isfinite(hip['e_RV'])]

hip['radial_velocity'] = hip['RV']
hip['radial_velocity_error'] = hip['e_RV']

hip = hip[hip['parallax'] > 0]
len(hip)

In [ ]:
g = GaiaData(hip)

In [ ]:
plt.hist(hip['phot_g_mean_mag'].value, bins=np.linspace(-5, 7, 64));
plt.yscale('log')

In [ ]:
# filename = '../data/hip_stars_near_theOG.csv'
# if os.path.exists(filename):
#     prev = at.QTable.read(filename, format='ascii.ecsv')
# else:
#     prev = None # For first run
prev = None # For first run

In [ ]:
c = g.get_skycoord()
the_og_c = the_og.get_skycoord()[0]

sep3d_mask = c.separation_3d(the_og_c) < 70*u.pc
vel3d_mask = (c.velocity - the_og_c.velocity).norm() < 15*u.km/u.s
xv_mask = vel3d_mask & sep3d_mask

subg = g[xv_mask]

if prev is not None:
    not_prev_mask = ~np.isin(subg.source_id, prev['source_id'])
    subg = subg[not_prev_mask]

sep3d_mask.sum(), vel3d_mask.sum(), xv_mask.sum(), len(subg)

In [ ]:
Cs = subg.get_cov(units=dict(pmra=u.mas/u.yr, pmdec=u.mas/u.yr, radial_velocity=u.km/u.s))
C = Cs[:, 2:, 2:]

In [ ]:
N = len(subg)

ra = subg.ra.to_value(u.rad)
dec = subg.dec.to_value(u.rad)
M = np.stack([get_tangent_basis(ra[i], dec[i]) for i in range(N)])
y = np.stack([subg.parallax.value, 
              subg.pmra.value, 
              subg.pmdec.value, 
              subg.radial_velocity.value], 
             axis=1)

Cinvs = np.stack([np.linalg.inv(C[n]) for n in range(N)], axis=0)

C.shape, M.shape, y.shape

In [ ]:
test_vxyz = subg.get_skycoord().velocity.d_xyz.value.T
test_d = subg.distance.value

In [ ]:
sigma_thin = 20.  # np.array([15, 35, 20.])
sigma_thik = 50.
sigma_halo = 125.  # np.full(3, 125.)  # km/s

# Number of prior mixture components:
K = 4
with pm.Model() as model:
    # r = pm.Uniform("r", 1, 500, shape=N, testval=test_d)
    r = UniformSpaceDensity("r", 2000, shape=N, testval=test_d)
    
    # Mixture model for 3D velocity
    w = pm.Dirichlet('w', a=np.ones(K))
    
    _v0 = pm.Normal.dist(v0, np.full(3, sigma_0), 
                         shape=(N, 3))
    _v1 = pm.Normal.dist(vthin, sigma_thin, 
                         shape=(N, 3))
    _v2 = pm.Normal.dist(vthik, sigma_thik, 
                         shape=(N, 3))
    _v3 = pm.Normal.dist(vhalo, sigma_halo, 
                         shape=(N, 3))
    vxyz = pm.Mixture('vxyz', w=w, comp_dists=[_v0, _v1, _v2, _v3], 
                      testval=test_vxyz, shape=(N, 3)) 
    
    v0_logp = pm.Deterministic('v0_logp', _v0.logp(vxyz))
    v1_logp = pm.Deterministic('v1_logp', _v1.logp(vxyz))
    v2_logp = pm.Deterministic('v2_logp', _v2.logp(vxyz))
    v3_logp = pm.Deterministic('v3_logp', _v3.logp(vxyz))
    
    # Velocity in tangent plane coordinates
    vtan = tt.batched_dot(M, vxyz)
    
    model_pm = vtan[:, :2] / r[:, None] * pc_mas_yr_per_km_s
    model_rv = vtan[:, 2:3]
    model_y = tt.concatenate((1000 / r[:, None], model_pm, model_rv), axis=1)
    
    # Option 3:
    dy = model_y - y
    val = -0.5 * tt.batched_dot(dy, tt.batched_dot(Cinvs, dy))

    pm.Potential("more_logp", val)

In [ ]:
with model:
    res = xo.optimize(start={'w': np.ones(K), 
                             'r': test_d,
                             'vxyz': test_vxyz})

In [ ]:
# res['r'] / test_d

In [ ]:
logps = np.zeros((N, K))
for k in range(K):
    logps[:, k] = res[f'v{k}_logp'].sum(axis=1)

In [ ]:
# for i in range(3):
#     _x = test_vxyz[:, i]
#     print(1.5 * np.median(np.abs(_x - np.median(_x))),
#           np.median(_x))
#     plt.hist(test_vxyz[:, i],
#              bins=np.linspace(-100, 100, 64), alpha=0.4, label=str(i),
#              density=True);
    
#     _grid = np.linspace(-100, 100, 1000)
#     plt.plot(_grid, norm.pdf(_grid, vdisk[i], sigma_disk[i]), marker='')
    
# plt.legend(loc='best')

In [ ]:
prob = np.exp(logps[:, 0] - logsumexp(logps, axis=1))
(prob > 0.5).sum()

In [ ]:
subg.source_id[prob > 0.5]

In [ ]:
tmp = hip[xv_mask].copy()
tmp['prob'] = prob
if prev is not None:
    all_tbl = at.QTable(at.vstack((at.Table(prev), at.Table(tmp))))
else:
    all_tbl = tmp
all_tbl.write('../data/hip_stars_near_theOG.csv', format='ascii.ecsv', overwrite=True)

---

OLD

In [ ]:
hip = at.QTable.read('/Users/apricewhelan/data/Hipparcos/hip2.fits')
hip = hip[hip['Plx'] > 0]

In [ ]:
brightbois = at.Table.read('/Users/apricewhelan/data/Hipparcos/Bright-star-catalog.fit')
brightbois = brightbois[np.isfinite(brightbois['RAJ2000'])]
bb_c = coord.SkyCoord(brightbois['RAJ2000'],
                      brightbois['DEJ2000'])

In [ ]:
idx, sep2d, _ = hip_c.match_to_catalog_sky(bb_c)

In [ ]:
hip['radial_velocity'] = np.full(len(hip), np.nan) * u.km/u.s
hip['radial_velocity'][sep2d < 10*u.arcsec] = brightbois['RadVel'][idx[sep2d < 10*u.arcsec]]

In [ ]:
hip_c = coord.SkyCoord(hip['RArad'], hip['DErad'],
                       distance=coord.Distance(parallax=hip['Plx']),
                       pm_ra_cosdec=hip['pmRA'],
                       pm_dec=hip['pmDE'],
                       radial_velocity=hip['radial_velocity'])

mask = ((hip_c.distance < 80*u.pc) & (hip_c.distance > 20*u.pc) &
        (hip_c.pm_ra_cosdec < -40*u.mas/u.yr) & 
        (hip_c.pm_dec > -90*u.mas/u.yr) & 
        (hip_c.pm_dec < 10*u.mas/u.yr) & 
        (hip_c.ra > 150*u.deg) & (hip_c.ra < 260*u.deg) &
        (hip_c.dec > 0*u.deg) & (hip_c.dec < 60*u.deg) &
        (hip['Hpmag'] < 7.5*u.mag))
mask.sum()

In [ ]:
hip_galcen = hip_c[mask].transform_to(coord.Galactocentric)
hip_v_mask = ((hip_galcen.velocity - np.mean(galcen.velocity)).norm() < 10*u.km/u.s)
hip_v_mask.sum()

In [ ]:
mhp = hip['Hpmag'] - hip_c.distance.distmod
BImag = hip['B-V']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.scatter(BImag[mask][hip_v_mask], mhp[mask][hip_v_mask])

for age in np.arange(7.4, 8.2+1e-3, 0.2):
    iso = at.Table.read(f'/Users/apricewhelan/data/Isochrones/MIST/FeH_0.1_iso.fits')
    i = iso[np.isclose(iso['log10_isochrone_age_yr'], age)] 
    i = i[i['phase'] < 4]  # rgb or before
    ax.plot(i['Tycho_B']-i['Tycho_V'], i['Hipparcos_Hp'], 
            marker='', label=f'log10age=${age:.1f}$', zorder=-10)

ax.set_xlim(-1, 2)
ax.set_ylim(7.5, -2)

In [ ]:
derp = hip[mask][hip_v_mask][BImag[mask][hip_v_mask] < 1*u.mag]
derp['HIP', 'Hpmag'][np.argsort(derp['Hpmag'])]